In [1]:
import os
os.environ['KB_AUTH_TOKEN'] = open('/tmp/kb_auth_token.txt').read().strip()

In [2]:
import biokbase
import biokbase.data_api.tests as tests
import biokbase.data_api.object
import biokbase.data_api.assembly
import biokbase.data_api.taxon
import biokbase.data_api.genome_annotation
import pandas as pd
import numpy as np
import qgrid as qg
qg.nbinstall()
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
%matplotlib inline

services = {
        "workspace_service_url": "https://ci.kbase.us/services/ws/",
        "shock_service_url": "https://ci.kbase.us/services/shock-api/",
    }

/Users/marcin/miniconda/envs/kbase-data-api/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
def get_genome_summary(genome_annotation=None):
    if genome_annotation == None:
        raise TypeError("No GenomeAnnotation object given.")
    elif genome_annotation.get_typestring().split('-')[0] not in biokbase.data_api.genome_annotation.TYPES:
        raise TypeError("{0} is not a recognized GenomeAnnotation type.".format(type(genome_annotation)))
    
    taxon = genome_annotation.get_taxon()
    assembly = genome_annotation.get_assembly()
    
    overview = dict()
    
    #get tax info
    taxon_details = dict()
    taxon_details["taxonomic_id"] = taxon.get_taxonomic_id()
    taxon_details["kingdom"] = taxon.get_kingdom()
    taxon_details["domain"] = taxon.get_domain()
    taxon_details["genetic_code"] = taxon.get_genetic_code()
    taxon_details["scientific_name"] = taxon.get_scientific_name()
    taxon_details["aliases"] = taxon.get_aliases()
    taxon_details["scientific_lineage"] = taxon.get_scientific_lineage()

    overview["taxon"] = taxon_details
    
    #get assembly info
    assembly_details = dict()
    assembly_details["number_of_contigs"] = assembly.get_number_contigs()
    assembly_details["total_length"] = assembly.get_dna_size()
    assembly_details["total_gc_content"] = assembly.get_gc_content()
    assembly_details["contig_length"] = assembly.get_contig_lengths()
    assembly_details["contig_gc_content"] = assembly.get_contig_gc_content()
    
    overview["assembly"] = assembly_details
    
    #get annotation info
    annotation_details = dict()
    annotation_details["feature_types"] = genome_annotation.get_feature_types()
    annotation_details["feature_type_descriptions"] = genome_annotation.get_feature_type_descriptions(annotation_details["feature_types"])
    annotation_details["feature_type_counts"] = genome_annotation.get_feature_type_counts(annotation_details["feature_types"])
    
    overview["annotation"] = annotation_details
    
    return overview

In [8]:
import pprint
import datetime

def parse_all_existing_annotations():
    annotations = dict()
    for x in object_list:
        if x.type.startswith("KBaseGenomes.Genome"):
            annotations[x.name] = x
    return annotations

b = biokbase.data_api.browse(1011)
object_list = b.ls()

test = b["kb|g.3157"]#object_list#

#annotations = {}
#for n in object_list:
#    annotations.update({n.name:n.object})

In [ ]:
from IPython.display import display, HTML

annotations_subset = {b[1].name: b[1].object, b[2].name: b[2].object}

overview = []
count = 0
for n in annotations_subset:
    print count
    print n
    #if count < 100 :
    over_now = get_genome_summary(annotations_subset[n])
    overview.append(over_now)
    #print over_now["assembly"]["number_of_contigs"]
    string_for_html = '<html><b>Genome</b>: ' + \
    over_now["taxon"]["scientific_name"] +\
    '<br><b>GC content</b>: ' + \
    `round(over_now["assembly"]["total_gc_content"],3)` +\
    '<br><b>Total DNA sequence length</b>: ' +\
    `("{:,}".format(over_now["assembly"]["total_length"])).replace("'","")` +\
    '<br><b>Number of contigs</b>:' +\
    `over_now["assembly"]["number_of_contigs"]` +\
    "</html>"
    display(HTML(string_for_html))
    #count = count +1
    #print count

0
kb|g.207120


Genome : Desulfovibrio zosterae DSM 11974 GC content : 0.418 Total DNA sequence length : '4,090,168' Number of contigs :19

In [10]:
b[1]

_ObjectInfo(objid=250, name=u'kb|g.3157', type=u'KBaseGenomes.Genome-8.0', save_date=u'2015-08-01T03:38:07+0000', version=1, saved_by=u'kbasetest', wsid=1011, workspace=u'OriginalReferenceGenomes', chsum=u'c58ec47b46dc348fa9841a80668d5a96', size=31731401, meta=None)